In [1]:
import pickle
from collections import Counter
from pathlib import Path

import pandas as pd

In [2]:
root = Path('/media/t9s9/SSD_ubuntu/datasets/SoundDescs/')
# extra_info.pkl only contains meta data about data format

with open(root / 'annotations/categories.pkl', 'rb') as f:
    cat = pickle.load(f)

with open(root / 'annotations/descriptions.pkl', 'rb') as f:
    desc = pickle.load(f)

In [5]:
df = pd.DataFrame({'category': cat, 'caption': desc}).reset_index(names='filename')
df = df.dropna(subset=['category', 'caption'])

df.filename = df.filename.apply(lambda x: str(Path('train') / Path(x).with_suffix('.flac')))
exists = df.filename.apply(lambda x: (root / 'data' / x).exists())
df = df[exists]

df.columns = df.columns.str.replace('category', 'tags')
df.columns = df.columns.str.replace('caption', 'captions')
df.captions = df.captions.apply(lambda x: [x])
df.to_parquet(root / 'annot.parquet', index=False)
df

,filename,tags,captions
0,train/nhu05030045.flac,[Nature],[King Penguin (Aptenodytes Patagonicus) - Colo...
1,train/07033129.flac,[Daily Life],[Railway Stations: Continuous Pa Anncts; Early...
2,train/07018018.flac,[Crowds],[Chatter - Crowd of 40-50 men & women in a hal...
3,train/nhu05062176.flac,[Nature],[Black-headed Wagtail (Motacilla Flava Feldegg...
4,train/nhu05047152.flac,[Nature],"[Garden - City garden during the day,summer. C..."
...,...,...,...
33057,train/07041019.flac,[Machines],[Chemical & Electrical Reactions - Fire exting...
33058,train/nhu05091204.flac,[Nature],"[Coniferous Forest Atmosphere - Wood warbler, ..."
33059,train/nhu05069044.flac,[Nature],[Western Sandpiper (Calidris Mauri) - AND SNOW...
33060,train/nhu05035062.flac,[Nature],[Robin (Erithacus Rubecula) - close-up spring/...


In [6]:
flat_cat = [cat for dp in df.tags.to_list() for cat in dp]
cat_counter = Counter(flat_cat)
cat_counter.most_common(10)

[('Nature', 17590),
 ('Transport', 3928),
 ('Machines', 2963),
 ('Daily Life', 2090),
 ('Military', 1097),
 ('Animals', 984),
 ('Clocks', 716),
 ('None', 707),
 ('Sport', 681),
 ('Footsteps', 617)]

In [7]:
df.iloc[2]

filename                                  train/07018018.flac
tags                                                 [Crowds]
captions    [Chatter - Crowd of 40-50 men & women in a hal...
Name: 2, dtype: object